# Entorno de trabajo

In [1]:
! wget https://datosabiertos.mineduc.cl/wp-content/uploads/2022/09/Directorio-oficial-EE-2022.rar
! wget https://datosabiertos.mineduc.cl/wp-content/uploads/2022/10/Resumen-matricula-EE-2022-2.rar
#! wget http://www.junaeb.cl/wp-content/uploads/2023/03/IVM_Establecimientos_2022.xlsx
! unrar e -r /content/Directorio-oficial-EE-2022.rar
! unrar e -r /content/Resumen-matricula-EE-2022-2.rar

--2023-10-01 21:36:45--  https://datosabiertos.mineduc.cl/wp-content/uploads/2022/09/Directorio-oficial-EE-2022.rar
Resolving datosabiertos.mineduc.cl (datosabiertos.mineduc.cl)... 163.247.44.239
Connecting to datosabiertos.mineduc.cl (datosabiertos.mineduc.cl)|163.247.44.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 828926 (809K)
Saving to: ‘Directorio-oficial-EE-2022.rar’

Directorio-oficial- 100%[===================>] 809.50K  1.02MB/s    in 0.8s    

2023-10-01 21:36:47 (1.02 MB/s) - ‘Directorio-oficial-EE-2022.rar’ saved [828926/828926]

--2023-10-01 21:36:47--  https://datosabiertos.mineduc.cl/wp-content/uploads/2022/10/Resumen-matricula-EE-2022-2.rar
Resolving datosabiertos.mineduc.cl (datosabiertos.mineduc.cl)... 163.247.44.239
Connecting to datosabiertos.mineduc.cl (datosabiertos.mineduc.cl)|163.247.44.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 755529 (738K)
Saving to: ‘Resumen-matricula-EE-2022-2.rar’

Resumen

In [2]:
! pip install -q git+https://github.com/h2oai/camelot
! pip install -q pdf2image
! pip install -q python-dateutil
! apt-get install poppler-utils
! pip install -q spacy
! python -m spacy download es_core_news_sm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 28.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.2 [186 kB]
Fetched 186 kB in 0s (433 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 120895 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.2_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.2) ...
Setting up poppler-utils (22.02.0-2ubuntu0.2) ...
Processing triggers for man-db (2.

In [3]:
import pandas as pd
import camelot
import ast
import spacy
import re
import unicodedata
nlp = spacy.load('es_core_news_sm')

In [4]:
def preprocesador(text):
    text = text.lower().strip()
    text = re.sub("\.", '', text)
    text = re.sub('ñ', 'ni', text)
    text = re.sub(' ', '_', text)
    text = re.sub('-', '_', text)
    text = re.sub(',', '', text)
    text = re.sub('\(', '', text)
    text = re.sub('\)', '', text)
    text = re.sub('/', '', text)
    text = re.sub('%', 'porcentaje', text)
    text = ''.join((c for c in unicodedata.normalize('NFD',text) if unicodedata.category(c) != 'Mn'))
    return text

# Directorio Oficial de Establecimientos 2022


## Metadata

In [5]:
pdf_file='/content/ER_Directorio_Oficial_EE_WEB.pdf'
variables_directorio_tablas = camelot.read_pdf(pdf_file, pages='2,3')
variables_directorio=pd.concat([
    variables_directorio_tablas[0].df.iloc[1::],
    variables_directorio_tablas[1].df.iloc[1::]
])
del variables_directorio_tablas
columnas=['Variable', 'Tipo', 'Descripcion', 'Valores']
variables_directorio.columns=columnas
variables_directorio['Descripcion']=variables_directorio['Descripcion'].str.replace('\n', '' )
variables_directorio['Descripcion']=variables_directorio['Descripcion'].str.replace('\t', '')
variables_directorio['Descripcion']=variables_directorio['Descripcion'].str.replace('\r', '')
variables_directorio['Valores']=variables_directorio['Valores'].str.replace('\n', ', ')

In [6]:
def extraer_diccionario(valores):
  if ':' in valores:
    valores=valores.replace('Administración , Delegada','Administración Delegada').replace('(o no , subvencionado)','(o no subvencionado)')
    try:
      diccionario=''
      for valor in [elem.split(':') for elem in valores.split(',')]:
        diccionario=diccionario+"'"+str(valor[0].strip())+"':'"+str(valor[1].strip()+"',")
      diccionario='{'+diccionario.rstrip(diccionario[-1])+'}'
      return str(diccionario)
    except:
        return str(valores)
  else:
    return str(valores)

In [7]:
variables_directorio['Variable']=variables_directorio['Variable'].apply(lambda x: x.replace('PAGO_MATRICULA6','PAGO_MATRICULA').replace('PAGO_MENSUAL7','PAGO_MENSUAL'))
variables_directorio['Valores']=variables_directorio['Valores'].apply(lambda x: extraer_diccionario(str(x)))
variables_directorio=variables_directorio.apply(lambda x: x.str.strip())

In [8]:
anexo1_directorio_tablas = camelot.read_pdf(pdf_file, pages='6,7')
anexo1_directorio=pd.concat([
    anexo1_directorio_tablas[0].df.iloc[1::],
    anexo1_directorio_tablas[1].df.iloc[1::]
])
del anexo1_directorio_tablas
diccionario_anexo1=''
for i in range(anexo1_directorio.shape[0]):
  diccionario_anexo1=diccionario_anexo1+"'"+anexo1_directorio[0].iloc[i]+"':'"+anexo1_directorio[1].iloc[i].replace('\n','')+"',"
diccionario_anexo1='{'+diccionario_anexo1.rstrip(diccionario_anexo1[-1])+'}'
anexo1_directorio[[2,1]].replace('',None).dropna()
anexo1_regiones_directorio=anexo1_directorio[[2,1]].replace('',None).dropna()
anexo1_regiones_directorio[1]=anexo1_regiones_directorio[1].apply(lambda x: x.replace('\n',''))
anexo1_provincias_directorio=anexo1_directorio[[3,4]].replace('',None).dropna()
anexo1_provincias_directorio[4]=anexo1_provincias_directorio[4].apply(lambda x: x.replace('\n',''))
diccionario_regiones_directorio=dict(zip(anexo1_regiones_directorio[2], anexo1_regiones_directorio[1]))
diccionario_provincias_directorio=dict(zip(anexo1_provincias_directorio[3], anexo1_provincias_directorio[4]))

In [9]:
anexo2_directorio_tablas = camelot.read_pdf(pdf_file, pages='8,9')
anexo2_directorio=pd.concat([
    anexo2_directorio_tablas[0].df.iloc[1::],
    anexo2_directorio_tablas[1].df[0].str.split("\n", expand = True).iloc[1::]
])
del anexo2_directorio_tablas
diccionario_anexo2=''
for i in range(anexo2_directorio.shape[0]):
  diccionario_anexo2=diccionario_anexo2+"'"+anexo2_directorio[0].iloc[i]+"':'"+anexo2_directorio[1].iloc[i].replace('\n','')+"',"
diccionario_anexo2='{'+diccionario_anexo2.rstrip(diccionario_anexo2[-1])+'}'

## Datos

In [10]:
df_directorio=pd.read_csv('20220914_Directorio_Oficial_EE_2022_20220430_WEB.csv', sep=';', dtype='str')

In [11]:
for columna in list(df_directorio.columns):
  valores=variables_directorio.loc[variables_directorio['Variable']==columna, 'Valores'].values[0]
  if '{' in valores:
    diccionario = ast.literal_eval(valores)
    df_directorio[columna]=df_directorio[columna].replace(diccionario)
  elif valores=='Ver Anexo I':
    if 'NOM_REG' in columna:
      variables_directorio.loc[variables_directorio['Variable']==columna, 'Valores']=str(diccionario_regiones_directorio)
      diccionario = diccionario_regiones_directorio
      df_directorio[columna]=df_directorio[columna].replace(diccionario)
    elif 'COD_PRO' in columna:
      variables_directorio.loc[variables_directorio['Variable']==columna, 'Valores']=str(diccionario_provincias_directorio)
      diccionario = diccionario_provincias_directorio
      df_directorio[columna]=df_directorio[columna].replace(diccionario)
    else:
      pass
  elif valores=='Ver Anexo II':
    variables_directorio.loc[variables_directorio['Variable']==columna, 'Valores']=str(diccionario_anexo2)
    diccionario = ast.literal_eval(diccionario_anexo2)
    df_directorio[columna]=df_directorio[columna].replace(diccionario)


In [12]:
variables_directorio.loc[variables_directorio['Variable']=='MAT_TOTAL', 'Tipo']='Numérico'
variables_directorio.loc[variables_directorio['Variable']=='MAT_TOTAL', 'Descripcion']='Matrícula Total del Establecimiento'
variables_directorio.loc[variables_directorio['Variable']=='LATITUD', 'Descripcion']='Latitud del Establecimiento'
variables_directorio.loc[variables_directorio['Variable']=='LONGITUD', 'Descripcion']='Longitud del Establecimiento'
variables_directorio.loc[variables_directorio['Variable']=='ENS_01', 'Descripcion']='Código de enseñanza 01'
variables_directorio.loc[variables_directorio['Variable']=='COD_COM_RBD', 'Descripcion']='Código oficial comuna en que se ubica el establecimiento'
variables_directorio.reset_index(inplace=True, drop=True)

In [13]:
def nombrar_columna(text):
  text.replace('\n','').replace('  ',' ')
  try:
    text = text.split('(')[0]
  except:
    text=text
  doc = nlp(text)
  tokens = [t.lemma_.lower() for t in doc if not t.is_punct | t.is_stop]
  return preprocesador('_'.join(tokens))

In [14]:
variables_directorio['Renombrado']=variables_directorio['Descripcion'].apply(lambda x: nombrar_columna(x))
variables_directorio.loc[variables_directorio['Variable']=='COD_DEPE', 'Renombrado']='dependencia_establecimiento'
variables_directorio.loc[variables_directorio['Variable']=='COD_DEPE2', 'Renombrado']='dependencia_agrupada_establecimiento'
variables_directorio.loc[variables_directorio['Variable']=='ESTADO_ESTAB', 'Renombrado']='estado_establecimiento'
variables_directorio.loc[variables_directorio['Variable']=='COD_PRO_RBD', 'Renombrado']='provincia_ubicacion_establecimiento'
variables_directorio.loc[variables_directorio['Variable']=='NOM_REG_RBD_A', 'Renombrado']='region_ubicacion_establecimiento'
for columna in list(df_directorio.columns):
  df_directorio.rename(columns={columna: variables_directorio.loc[variables_directorio['Variable']==columna, 'Renombrado'].values[0]}, inplace=True)


In [15]:
variables_directorio

,Variable,Tipo,Descripcion,Valores,Renombrado
0,AGNO,Numérico,Año,,anio
1,RBD,Numérico,Rol base de datos del establecimiento,,rol_base_dato_establecimiento
2,DGV_RBD,Numérico,Dígito verificador del RBD,,digito_verificador_rbd
3,NOM_RBD,Cadena,Nombre del establecimiento,,nombre_establecimiento
4,LET_RBD,Cadena,Letra del establecimiento,,letra_establecimiento
5,NUM_RBD,Numérico,Número del establecimiento.,,numero_establecimiento
6,MRUN,Numérico,Máscara del RUN del sostenedor (sólo aplica a ...,,mascara_run_sostenedor
7,RUT_SOSTENEDOR,Numérico,RUT del sostenedor del establecimiento (sólo a...,,rut_sostenedor_establecimiento
8,P_JURIDICA,Numérico,Indicador si el sostenedor corresponde a una p...,"{'0':'No (Persona Natural)','1':'Sí','9':'Sin ...",indicador_sostenedor_corresponder_persona_juri...
9,COD_REG_RBD,Numérico,Código de región en que se ubica el establecim...,Ver Anexo I,codigo_region_ubicar_establecimiento


In [16]:
df_directorio

,anio,rol_base_dato_establecimiento,digito_verificador_rbd,nombre_establecimiento,mascara_run_sostenedor,rut_sostenedor_establecimiento,indicador_sostenedor_corresponder_persona_juridica,codigo_region_ubicar_establecimiento,region_ubicacion_establecimiento,provincia_ubicacion_establecimiento,...,codigo_ensenianza_09,codigo_ensenianza_10,codigo_ensenianza_11,matricula_establecimiento,establecimiento_matricula_30_abril,estado_establecimiento,orientacion_religiosa_establecimiento,detalle_orientacion_religiosa_establecimiento,pago_matricula_establecimiento,pago_mensualidad_establecimiento
0,2022,1,9,LICEO POLITECNICO ARICA,,62000660,Sí,15,Región de Arica y Parinacota,Provincia de Arica,...,No Aplica,No Aplica,No Aplica,667,Sí,Funcionando,Orientación religiosa Católica,,GRATUITO,GRATUITO
1,2022,2,7,PARVULARIO LAS ESPIGUITAS,,62000660,Sí,15,Región de Arica y Parinacota,Provincia de Arica,...,No Aplica,No Aplica,No Aplica,166,Sí,Funcionando,Orientación religiosa Católica,,GRATUITO,GRATUITO
2,2022,3,5,ESC. PEDRO VICENTE GUTIERREZ TORRES,,62000660,Sí,15,Región de Arica y Parinacota,Provincia de Arica,...,No Aplica,No Aplica,No Aplica,376,Sí,Funcionando,Orientación religiosa Laica,,GRATUITO,GRATUITO
3,2022,4,3,LICEO OCTAVIO PALMA PEREZ,,62000660,Sí,15,Región de Arica y Parinacota,Provincia de Arica,...,No Aplica,No Aplica,No Aplica,1014,Sí,Funcionando,Orientación religiosa Laica,,GRATUITO,GRATUITO
4,2022,5,1,JOVINA NARANJO FERNANDEZ,,62000660,Sí,15,Región de Arica y Parinacota,Provincia de Arica,...,No Aplica,No Aplica,No Aplica,771,Sí,Funcionando,Orientación religiosa Otro,PLURALISTA,GRATUITO,GRATUITO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16596,2022,42089,1,SALA CUNA Y JARDIN INFANTIL COLORÍN COLORADO,,76226310,Sí,10,Región de Los Lagos,Provincia de Llanquihue,...,No Aplica,No Aplica,No Aplica,0,No,Autorizado sin matrícula,Sin información,,SIN INFORMACION,SIN INFORMACION
16597,2022,42090,5,ESCUELA HOSPITALARIA CLUB DE LEONES,,65291980,Sí,11,Región de Aysén del Gral. Carlos Ibáñez del Campo,Provincia de Coyhaique,...,No Aplica,No Aplica,No Aplica,0,No,Funcionando,Orientación religiosa Laica,,GRATUITO,GRATUITO
16598,2022,42091,3,SALA CUNA RAYUN,,61607400,Sí,9,Región de la Araucanía,Provincia de Cautín,...,No Aplica,No Aplica,No Aplica,0,No,Autorizado sin matrícula,Sin información,,SIN INFORMACION,SIN INFORMACION
16599,2022,42092,1,HEIDI,,61607400,Sí,9,Región de la Araucanía,Provincia de Cautín,...,No Aplica,No Aplica,No Aplica,0,No,Autorizado sin matrícula,Sin información,,SIN INFORMACION,SIN INFORMACION


In [17]:
columnas=[
        'anio',
        'rol_base_dato_establecimiento',
        'digito_verificador_rbd',
        'nombre_establecimiento',
        'mascara_run_sostenedor',
        'rut_sostenedor_establecimiento',
        'indicador_sostenedor_corresponder_persona_juridica',
        #'codigo_region_ubicar_establecimiento',
        'region_ubicacion_establecimiento',
        'provincia_ubicacion_establecimiento',
        #'codigo_oficial_comuna_ubicar_establecimiento',
        'nombre_comuna_establecimiento',
        #'codigo_departamento_provincial_educacion_pertenecer_establecimiento',
        'nombre_departamento_provincial_educacion_pertenecer_establecimiento',
        #'dependencia_establecimiento',
        'dependencia_agrupada_establecimiento',
        'indicador_ruralidad',
        'latitud_establecimiento',
        'longitud_establecimiento',
        'indicador_establecimiento_convenio_pie',
        'indicador_establecimiento_programa_pace',
        'codigo_ensenianza_01',
        'codigo_ensenianza_02',
        'codigo_ensenianza_03',
        'codigo_ensenianza_04',
        'codigo_ensenianza_05',
        'codigo_ensenianza_06',
        'codigo_ensenianza_07',
        'codigo_ensenianza_08',
        'codigo_ensenianza_09',
        'codigo_ensenianza_10',
        'codigo_ensenianza_11',
        'matricula_establecimiento',
        'establecimiento_matricula_30_abril',
        'estado_establecimiento',
        'orientacion_religiosa_establecimiento',
        'detalle_orientacion_religiosa_establecimiento',
        'pago_matricula_establecimiento',
        'pago_mensualidad_establecimiento'
        ]

In [18]:
df_directorio.loc[df_directorio['provincia_ubicacion_establecimiento']=='Provincia de Chiloé'][columnas].to_csv('df_directorio.csv', index=False)

# Resumen de Matrícula Oficial por Establecimientos 2022

In [19]:
columnas_interes=[
    'RBD',
    'MAT_ENS_1',
    'MAT_ENS_2',
    'MAT_ENS_3',
    'MAT_ENS_4',
    'MAT_ENS_5',
    'MAT_ENS_6',
    'MAT_ENS_7',
    'MAT_ENS_8',
    'MAT_TOTAL',
    'CUR_SIM_01',
    'CUR_SIM_02',
    'CUR_SIM_03',
    'CUR_SIM_04',
    'CUR_SIM_05',
    'CUR_SIM_06',
    'CUR_SIM_07',
    'CUR_SIM_08',
    'CUR_SIM_TOT',
    'CUR_COMB_01',
    'CUR_COMB_02',
    'CUR_COMB_TOT'
]

In [20]:
df_matricula=pd.read_csv('20221013_Resumen_Matrícula_EE_Oficial_2022_20220430_WEB.csv', sep=';')

In [21]:
df_matricula.columns

Index(['AGNO', 'RBD', 'DGV_RBD', 'NOM_RBD', 'COD_DEPE', 'COD_DEPE2',
       'RURAL_RBD', 'COD_REG_RBD', 'NOM_REG_RBD_A', 'COD_PRO_RBD',
       'COD_COM_RBD', 'NOM_COM_RBD', 'COD_DEPROV_RBD', 'NOM_DEPROV_RBD',
       'ESTADO_ESTAB', 'MAT_SI_1', 'MAT_HOM_1', 'MAT_MUJ_1', 'MAT_ENS_1',
       'MAT_SI_2', 'MAT_HOM_2', 'MAT_MUJ_2', 'MAT_ENS_2', 'MAT_HOM_3',
       'MAT_MUJ_3', 'MAT_ENS_3', 'MAT_SI_4', 'MAT_HOM_4', 'MAT_MUJ_4',
       'MAT_ENS_4', 'MAT_SI_5', 'MAT_HOM_5', 'MAT_MUJ_5', 'MAT_ENS_5',
       'MAT_HOM_6', 'MAT_MUJ_6', 'MAT_ENS_6', 'MAT_HOM_7', 'MAT_MUJ_7',
       'MAT_ENS_7', 'MAT_HOM_8', 'MAT_MUJ_8', 'MAT_ENS_8', 'MAT_HOM_TOT',
       'MAT_MUJ_TOT', 'MAT_SI_TOT', 'MAT_TOTAL', 'CUR_SIM_01', 'CUR_SIM_02',
       'CUR_SIM_03', 'CUR_SIM_04', 'CUR_SIM_05', 'CUR_SIM_06', 'CUR_SIM_07',
       'CUR_SIM_08', 'CUR_SIM_TOT', 'CUR_COMB_01', 'CUR_COMB_02',
       'CUR_COMB_TOT'],
      dtype='object')

# Servicio (streamlit)

In [22]:
! pip install -q 'pygwalker==0.3.5'
! pip install -q plotly
! pip install -q 'streamlit>=1.8.1'
! pip install -q pandas-profiling
! pip install -q streamlit-pandas-profiling
! pip install -q click==8
! npm install localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.3/313.3 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.3/357.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 12.5 MB/s eta 0:00:

In [23]:
%%writefile app.py
import pygwalker as pyg
import pandas as pd
import ydata_profiling
import plotly.express as px
import streamlit as st
import streamlit.components.v1 as components
from streamlit_pandas_profiling import st_profile_report
from pandas.api.types import (
    is_categorical_dtype,
    is_datetime64_any_dtype,
    is_numeric_dtype,
    is_object_dtype,
)
df = pd.read_csv('df_directorio.csv', dtype='category')
pr = df.profile_report(minimal=True, title='Establecimientos Chiloé').to_html()
def main():
  global df
  global pr
  st.set_page_config(
      layout="wide"
  )
  @st.cache_resource
  def convert_df(df):
    return df.to_csv(index=False).encode('utf-8')
  def filter_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds a UI on top of a dataframe to let viewers filter columns

    Args:
        df (pd.DataFrame): Original dataframe

    Returns:
        pd.DataFrame: Filtered dataframe
    """
    modify = st.checkbox("Agregar filtros")

    if not modify:
        return df
    df = df.copy()
    for col in df.columns:
        if is_object_dtype(df[col]):
            try:
                df[col] = pd.to_datetime(df[col])
            except Exception:
                pass
        if is_datetime64_any_dtype(df[col]):
            df[col] = df[col].dt.tz_localize(None)
    modification_container = st.container()
    with modification_container:
        to_filter_columns = st.multiselect("Filtrar datos según ", df.columns)
        for column in to_filter_columns:
            left, right = st.columns((1, 20))
            # Treat columns with < 10 unique values as categorical
            if is_categorical_dtype(df[column]) or df[column].nunique() < 10:
                user_cat_input = right.multiselect(
                    f"Valores para {column}",
                    df[column].unique(),
                    default=list(df[column].unique()),
                )
                df = df[df[column].isin(user_cat_input)]
            elif is_numeric_dtype(df[column]):
                _min = float(df[column].min())
                _max = float(df[column].max())
                step = (_max - _min) / 100
                user_num_input = right.slider(
                    f"Valores para {column}",
                    min_value=_min,
                    max_value=_max,
                    value=(_min, _max),
                    step=step,
                )
                df = df[df[column].between(*user_num_input)]
            elif is_datetime64_any_dtype(df[column]):
                user_date_input = right.date_input(
                    f"Valores para {column}",
                    value=(
                        df[column].min(),
                        df[column].max(),
                    ),
                )
                if len(user_date_input) == 2:
                    user_date_input = tuple(map(pd.to_datetime, user_date_input))
                    start_date, end_date = user_date_input
                    df = df.loc[df[column].between(start_date, end_date)]
            else:
                user_text_input = right.text_input(
                    f"Substring or regex in {column}",
                )
                if user_text_input:
                    df = df[df[column].astype(str).str.contains(user_text_input)]

    return df

  df['latitud_establecimiento'] = pd.to_numeric(df['latitud_establecimiento'].apply(lambda x: x.replace(',','.').replace(' ','')), errors='coerce').astype('float')
  df['longitud_establecimiento'] = pd.to_numeric(df['longitud_establecimiento'].apply(lambda x: x.replace(',','.').replace(' ','')), errors='coerce').astype('float')
  df['matricula_establecimiento'] = pd.to_numeric(df['matricula_establecimiento'], errors='coerce').astype('int')
  st.title("Fundación Tierra de Gaviotas - Observatorio Establecimientos Chiloé")
  csv = convert_df(df)
  st.write(df.astype('object'))
  st.download_button(
   "Descargar .csv establecimientos Chiloé",
   csv,
   "establecimientos_chiloe.csv",
   "text/csv",
   key='download-csv'
  )
  st.map(df[['latitud_establecimiento','longitud_establecimiento']].dropna(),
    latitude='latitud_establecimiento',
    longitude='longitud_establecimiento')
  #st_profile_report(pr)
  components.html(pr, height=5000, scrolling=True)
  st.title("Servicio de análisis")
  st.write('Visualización')
  df_filtrar = df.copy()
  df=df.rename(columns={
                          'latitud_establecimiento':'latitude',
                          'longitud_establecimiento':'longitude'
    })
  pyg_html = pyg.walk(df, return_html=True)
  components.html(pyg_html, height=1000, scrolling=True)
  st.write('Filtrado')
  df_filtrado=filter_dataframe(df_filtrar)
  st.dataframe(df_filtrado)
  csv_filtrado = convert_df(df_filtrado)
  st.download_button(
   "Descargar .csv establecimientos filtrados",
   csv_filtrado,
   "establecimientos_filtrados.csv",
   "text/csv",
   key='download-csv-filtrado'
  )
  fig = px.scatter_geo(df_filtrado,
                    lat='latitud_establecimiento',
                    lon='longitud_establecimiento',
                    color='dependencia_agrupada_establecimiento',
                    hover_name='nombre_establecimiento')
  fig.update_geos(fitbounds="locations")
  fig.update_geos(resolution=50)
  st.plotly_chart(fig, use_container_width=True)

if __name__ == "__main__":
    main()

Writing app.py


In [24]:
!curl ipecho.net/plain

34.83.109.216

In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipecho.net/plain